In [ ]:
import pandas as pd 
import numpy as np
from pylatex import Document, Tabular, utils, Command

In [ ]:
data_src = r'../dat/bundesnetzagentur/pre_processed_data/prod_price_de_lu_2019_2021.csv'

In [ ]:
df = pd.read_csv(data_src,
                parse_dates=['time'])
df = df.set_index('time')
df = df.resample('Y').mean()

renewable = ['Biomass', 'Hydropower', 'Solar', 'Wind', 'Other Renewables']
conventional = ['Nuclear', 'Brown Coal', 'Black Coal' ,'Natural Gas', 'Other Conventional Sources']


price = df['Price']
df.drop(['Pumped-Storage Hydro', 'Price', 'Solar & Wind', 'Offshore Wind', 'Onshore Wind'], axis=1, inplace=True)

df['Total renewable'] = df[renewable].sum(axis=1)
df['Total conventional'] = df[conventional].sum(axis=1)
df['Total'] = df[['Total renewable', 'Total conventional']].sum(axis=1)
df = df.round().astype(int)
df.head()

In [ ]:
# function for adding new rows to the tabular
def add_row(tabular, tabular_row_name, df, df_column_name):
    tabular.add_row((tabular_row_name, df[df_column_name].values[0], df[df_column_name].values[1], df[df_column_name].values[2]))

In [ ]:
doc = Document()
tabular = Tabular(table_spec='l c c c',
                  col_space='12pt',
                  booktabs=True)

with doc.create(tabular) as table:
    table.add_row(('', utils.bold('2019'), utils.bold('2020'), utils.bold('2021')))
    table.add_hline()
    table.add_row((utils.italic('Renewable'), '', '', ''))
    add_row(table, 'Wind', df, 'Wind')
    add_row(table, 'Hydropower', df, 'Hydropower')
    add_row(table, 'Solar', df, 'Solar')
    add_row(table, 'Biomass', df, 'Biomass')
    add_row(table, 'Other Renewables', df, 'Other Renewables')
    table.add_hline()
    add_row(table, 'Total Renewables', df, 'Total renewable')
    table.add_empty_row()
    table.add_row((utils.italic('Conventional'), '', '', ''))
    add_row(table, 'Nuclear', df, 'Nuclear')
    add_row(table, 'Brown Coal', df, 'Brown Coal')
    add_row(table, 'Black Coal', df, 'Black Coal')
    add_row(table, 'Natural Gas', df, 'Natural Gas')
    add_row(table, 'Other Conventionals', df, 'Other Conventional Sources')
    table.add_hline()
    add_row(table, 'Total Conventionals', df, 'Total conventional')
    table.add_empty_row()
    table.add_hline()
    add_row(table, utils.bold('Total'), df, 'Total')

doc.generate_pdf('comparison_table', clean_tex=False)